#Model development

##Install packages

In [1]:
pip install pandas scikit-learn tensorflow keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.1 MB/s eta 0:00:00


##Import libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from google.colab import drive

<ipython-input-2-3f0f5481e0d1>:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


##Data split

In [3]:
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/sonar_updated.csv')
X = data.drop('60', axis=1)
Y = data['60']

encoder = LabelEncoder()
Y = to_categorical(encoder.fit_transform(Y))

X_train, X_sec, Y_train, Y_sec = train_test_split(X, Y, test_size=0.4, random_state=0)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_sec, Y_sec, test_size=0.5, random_state=0)

Mounted at /content/drive


##Neural network model

In [4]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units_input', min_value=32, max_value=512, step=32),
        input_dim=X_train.shape[1],
        activation='relu'
    ))
    for i in range(hp.Int('n_layers', 1, 5)):
        model.add(Dense(
            units=hp.Int(f'units_layer{i}', min_value=32, max_value=512, step=32),
            activation='relu'
        ))
    model.add(Dense(2, activation='softmax'))
    model.compile(
        optimizer=Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

##Hyperparemeter tuning

In [5]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='sonar_kt'
)

tuner.search(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))

best_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.630952388048172

Best val_accuracy So Far: 0.8095238208770752
Total elapsed time: 00h 00m 54s


##Model training

In [6]:
best_model.fit(X_train, Y_train, epochs=50, batch_size=10, validation_split=0.2)
history = best_model.fit(X_test, Y_test, epochs=50, batch_size=10, validation_split=0.2)
loss, accuracy = best_model.evaluate(X_test, Y_test)
print(f'Best model loss: {loss}, accuracy: {accuracy}')

tuner.results_summary()

Epoch 1/50
10/10 [==============================] - 1s 63ms/step - loss: 0.6220 - accuracy: 0.6566 - val_loss: 0.4243 - val_accuracy: 0.8800
Epoch 2/50
10/10 [==============================] - 0s 28ms/step - loss: 0.5726 - accuracy: 0.7374 - val_loss: 0.4545 - val_accuracy: 0.8400
Epoch 3/50
10/10 [==============================] - 0s 15ms/step - loss: 0.5532 - accuracy: 0.6869 - val_loss: 0.5091 - val_accuracy: 0.7200
Epoch 4/50
10/10 [==============================] - 0s 16ms/step - loss: 0.4562 - accuracy: 0.7778 - val_loss: 0.4352 - val_accuracy: 0.7200
Epoch 5/50
10/10 [==============================] - 0s 15ms/step - loss: 0.4334 - accuracy: 0.7677 - val_loss: 0.4354 - val_accuracy: 0.8400
Epoch 6/50
10/10 [==============================] - 0s 21ms/step - loss: 0.4221 - accuracy: 0.7677 - val_loss: 0.4215 - val_accuracy: 0.8400
Epoch 7/50
10/10 [==============================] - 0s 21ms/step - loss: 0.4071 - accuracy: 0.7980 - val_loss: 0.3927 - val_accuracy: 0.7600
Epoch 8/50
10